In [1]:
import tensorflow as tf
import numpy as np
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from wrappers import wrapper
from agent_testing import DQNAgent


env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, RIGHT_ONLY)
env = wrapper(env)

states = (84, 84, 4)
actions = env.action_space.n

agent = DQNAgent(states=states, actions=actions, max_memory=100000, double_q=True)
agent.reloadModel()



Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
INFO:tensorflow:Restoring parameters from ./models/model-2000000


In [2]:
for i in range(100):
    state = env.reset()
    total_reward = 0
    while True:
        env.render()
        action = agent.run(state=state)
        next_state, reward, done, info = env.step(action=action)
        total_reward += reward
        state = next_state
        if done:
            break

/home/gmo/.local/lib/python3.5/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


KeyboardInterrupt: 